In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
import os

## Setup BuildingMOTIF

In [2]:
file = "./test.db"
if os.path.isfile(file):
    os.remove(file) 

building_motif = BuildingMOTIF(f"sqlite:///{file}")
building_motif.setup_tables()

## Load model from file

In [3]:
medium_office_model = Model.create("http://example.org/building/")
medium_office_model.graph.parse(os.path.join('mediumOffice-validation', 'mediumOffice_brick.ttl'), format="ttl")

<Graph identifier=711c8a7b-0328-4a2a-a5d5-4d566348f8ba (<class 'rdflib.graph.Graph'>)>

In [4]:
brick = Library.load(ontology_graph="../libraries/brick/Brick-full.ttl")
ashrae_g36 = Library.load(directory="../libraries/ashrae/guideline36/")


2022-07-27 12:09:31,048 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2022-07-27 12:09:31,049 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice


In [5]:
manifest = Library.load(directory="mediumOffice-validation/constraints/")

## Validate that model passes constraints

In [6]:
shape_collections = [brick.get_shape_collection()]
validation_result, report_text, new_graph = medium_office_model.validate(shape_collections)
print(report_text)
old_graph = medium_office_model.graph
medium_office_model.graph = new_graph
validation_result, report_text, new_graph = medium_office_model.validate([manifest.get_shape_collection()])
print(report_text)
medium_office_model.graph = old_graph


Validation Report
Conforms: True

Validation Report
Conforms: False
Results (18):
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:feeds ; sh:qualifiedMaxCount Literal("1", datatype=xsd:integer) ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:class brick:HVAC_Zone ] ]
	Focus Node: <http://example.org/building/Core_mid-ZN-VAV-Terminal>
	Result Path: brick:feeds
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:path brick:feeds ; sh:qualifiedMaxCount Literal("1", datatype=xsd:integer) ; sh:qualifiedMinCount Literal("1", datatype=xsd:integer) ; sh:qualifiedValueShape [ sh:class brick:HVAC_Zone ] ]
	Focus Node: <http://example.org/building/Core_top-ZN-VAV-Terminal>
	Result Path: brick:feeds
Cons

In [11]:
print((new_graph - old_graph).serialize(format="ttl"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<http://example.org/building/5-Zone-PVAV> brick:hasTag <https://brickschema.org/schema/BrickTag#AHU>,
        <https://brickschema.org/schema/BrickTag#Equipment> .

<http://example.org/building/5-Zone-PVAV-1> brick:hasTag <https://brickschema.org/schema/BrickTag#AHU>,
        <https://brickschema.org/schema/BrickTag#Equipment> .

<http://example.org/building/5-Zone-PVAV-1-2spd-DX-Clg-Coil-360kBtu/hr-9.8EER> brick:hasTag <https://brickschema.org/schema/BrickTag#Coil>,
        <https://brickschema.org/schema/BrickTag#Cool>,
        <https://brickschema.org/schema/BrickTag#Equipment> .

<http://example.org/building/5-Zone-PVAV-1-2spd-DX-Clg-Coil-360kBtu/hr-9.8EER-Outlet-Air-Node_System_Node_Mass_Flow_Rate> brick:hasTag <https://brickschema.org/schema/BrickTag#Air>,
        <https://brickschema.org/schema/BrickTag#Discharge>,
        <https://brickschema.org/schema/BrickTag#Flow>,
        <https://brickschema.org/schema/BrickTag#Poi

## Fix Model

In [ ]:
# Model is missing feeds from 1 AHU to 1 VAV

# 1 VAV is missing Discharge_Air_Flow_Sensor
# apply g36 template to that VAV

## Validate and pass